<a href="https://colab.research.google.com/github/DevBhuyan/Image-classification/blob/master/Sign_Language_MultiClass_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ATTENTION: Please do not alter any of the provided code in the exercise. Only add your own code where indicated
# ATTENTION: Please do not add or remove any cells in the exercise. The grader will check specific cells based on the cell position.
# ATTENTION: Please use the provided epoch values when training.

import csv
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from os import getcwd

In [2]:
def get_data(filename):
  # You will need to write code that will read the file passed
  # into this function. The first line contains the column headers
  # so you should ignore it
  # Each successive line contians 785 comma separated values between 0 and 255
  # The first value is the labels
  # The rest are the pixel values for that picture
  # The function will return 2 np.array types. One with all the labels
  # One with all the images
  #
  # Tips: 
  # If you read a full line (as 'row') then row[0] has the label
  # and row[1:785] has the 784 pixel values
  # Take a look at np.array_split to turn the 784 pixels into 28x28
  # You are reading in strings, but need the values to be floats
  # Check out np.array().astype for a conversion
    with open(filename) as training_file:
      # Your code starts here
        #lbls = []
        labels = []
        #imgs = []
        images = []
        pixels = []
        ct = 0
        for line in training_file:
            ct += 1
            if ct == 1:
                continue
            pixel_nums = line.split(',')
            #for i in pixel_nums: pixels.append(float(i))
            np_img = np.asarray(pixel_nums[1:], dtype = 'float')
            img_rows = np.array_split(np_img, 28)
            #lbls.append(pixels[0])
            #imgs.append(img_rows)
            labels.append(float(pixel_nums[0]))
            images.append(img_rows)
        #labels = np.asarray(lbls)
        #images = np.asarray(imgs)
        print(max(labels))
      # Your code ends here
    return images, labels

path_sign_mnist_train = f"{getcwd()}/../tmp2/sign_mnist_train.csv"
path_sign_mnist_test = f"{getcwd()}/../tmp2/sign_mnist_test.csv"
#training_images, training_labels = get_data(path_sign_mnist_train)
#testing_images, testing_labels = get_data(path_sign_mnist_test)

tr_img_list, tr_lb_list = get_data(path_sign_mnist_train)
t_img_list, t_lb_list = get_data(path_sign_mnist_test)
training_images = np.asarray(tr_img_list)
training_labels = np.asarray(tr_lb_list)
testing_images = np.asarray(t_img_list)
testing_labels = np.asarray(t_lb_list)

# Keep these
print(training_images.shape)
print(training_labels.shape)
print(testing_images.shape)
print(testing_labels.shape)

print(training_images)
print(training_labels)
print(testing_images)
print(testing_labels)

# Their output should be:
# (27455, 28, 28)
# (27455,)
# (7172, 28, 28)
# (7172,)

FileNotFoundError: ignored

In [ ]:
# In this section you will have to add another dimension to the data
# So, for example, if your array is (10000, 28, 28)
# You will need to make it (10000, 28, 28, 1)
# Hint: np.expand_dims

training_images = np.expand_dims(training_images, axis = 3)
testing_images = np.expand_dims(testing_images, axis = 3)

# Create an ImageDataGenerator and do Image Augmentation
train_datagen = ImageDataGenerator(
    # Your Code Here
    rescale = 1/255,
    horizontal_flip = True,
    rotation_range = 40,
    width_shift_range = 0.2,
    shear_range = 0.2,
    zoom_range = 0.2,
    fill_mode = 'nearest'
    )

validation_datagen = ImageDataGenerator(
    # Your Code Here
    rescale = 1/255
    )
    
train_datagen.fit(training_images)
validation_datagen.fit(testing_images)
    
# Keep These
print(training_images.shape)
print(testing_images.shape)
    
# Their output should be:
# (27455, 28, 28, 1)
# (7172, 28, 28, 1)

In [ ]:
# Define the model
# Use no more than 2 Conv2D and 2 MaxPooling2D
model = tf.keras.models.Sequential([
    # Your Code Here
    keras.layers.Conv2D(32, (3, 3), activation = 'relu', input_shape = (28, 28, 1)),
    keras.layers.MaxPooling2D(2, 2),
    keras.layers.Conv2D(64, (3, 3), activation = 'relu'),
    keras.layers.MaxPooling2D(2, 2),
    keras.layers.Flatten(),
    #keras.layers.Dropout(0.5),
    keras.layers.Dense(512, activation = 'relu'),
    keras.layers.Dense(26, activation = 'softmax')
    ])

# Compile Model. 
model.compile(# Your Code Here
                loss = 'sparse_categorical_crossentropy',
                optimizers = 'RMSprop',
                metrics = ['accuracy'])

# Train the Model
history = model.fit_generator(# Your Code Here (set 'epochs' = 2)
                                train_datagen.flow(training_images, training_labels, batch_size = 5),
                                epochs = 2,
                                steps_per_epoch = len(training_images)/5,
                                validation_data = validation_datagen.flow(testing_images, testing_labels, batch_size = 5),
                                validation_steps = len(testing_images)/5,
                                verbose = 2)

model.evaluate(testing_images, testing_labels, verbose=0)

In [ ]:
# Plot the chart for accuracy and loss on both training and validation
%matplotlib inline
import matplotlib.pyplot as plt
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()

plt.plot(epochs, loss, 'r', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()